# COSTO UNITARIO PROMEDIO

### OBTENCION COSTO DEL PRODUCTO

In [1]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id', 'product_id', 'product_qty']}
)

# 2. Extraer los productos únicos de esas órdenes
ids_productos_manu = list({op['product_id'][0] for op in ordenes_produccion if op.get('product_id')})

# 3. Leer los productos manufacturados que correspondan a DONSSON
productos_manu = models.execute_kw(
    db, uid, password,
    'product.product', 'read',
    [ids_productos_manu],
    {'fields': ['id', 'name', 'standard_price', 'catalogo_codigo_fab', 'qty_available']}
)

# 4. Filtrar solo marca DONSSON
productos_donsson_manu = [p for p in productos_manu if p.get('catalogo_codigo_fab') == 'DONSSON']



productos_manu = pd.DataFrame(productos_manu) # productos con costo entre los manufacturados


In [2]:
productos_manu = productos_manu.rename(columns={
    'name': 'producto',
    'standard_price': 'costo_estandar_promedio',
    'qty_available': 'cantidad_disponible'
})

pd.set_option("display.max_colwidth", None)
productos_manu = productos_manu[["producto","costo_estandar_promedio","cantidad_disponible"]]

#### Buscador para casos especiales productos

In [3]:
productos_manu[productos_manu["producto"].str.contains("DA2919", case=False, na=False)]

,producto,costo_estandar_promedio,cantidad_disponible
292,DA2919 FILTROAIRE CHEV. LV150,45119.8,116.0


### OBTENCION COSTO DE ORDEN DE PRODUCCION

In [4]:
# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)



ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

In [5]:
ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12113 entries, 0 to 12112
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   valor_unitario_mrp  12113 non-null  float64
 1   product_id          12113 non-null  object 
 2   date_finished       12113 non-null  object 
 3   product_qty         12113 non-null  float64
 4   id                  12113 non-null  int64  
 5   name                12113 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 567.9+ KB


In [6]:
ops.head()

,valor_unitario_mrp,product_id,date_finished,product_qty,id,name
0,7481.39,"[17320, [DAB02995025] DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD (025 DA2995)]",2018-03-28 13:00:43,1001.0,2388,O02445
1,22989.87,"[17938, [DAB08080025] DA8080 FILTRO AIRE BOBCAT (025 DA8080)]",2025-09-30 09:06:06,30.0,13527,MO13257
2,23631.04,"[17038, [DAB02547025] DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E (025 DA2547)]",2025-09-30 15:27:57,700.0,13513,MO13243
3,23985.10,"[17262, [DAB02932025] DA2932 FILTRO AIRE - MASSEY FERGUSON - IVECO (025 DA2932)]",2025-09-30 13:31:51,120.0,13507,MO13237
4,56186.35,"[17138, [DAB12728025] DA2728A FILTROAIRE 1º TEREX. (025 DA2728A)]",2025-09-30 13:30:46,80.0,13505,MO13235


In [7]:
ops = ops.rename(columns={

    "name" :"nombre_op",
    "product_qty":"unidades_producidas",
    "valor_unitario_mrp":"costo_unitario_op",
    "date_finished":"fecha_cierre_op"
})


In [8]:
import re

# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)




#### Buscador casos especiales por op

In [9]:
ops = ops[["producto","unidades_producidas","nombre_op","costo_unitario_op","fecha_cierre_op"]]
pd.set_option("display.max_colwidth", None)

ops[ops["producto"].str.contains("DA2983", case=False, na=False)]



,producto,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op
127,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO13059,10804.43,2025-09-12 13:51:18
740,DA2983 FILTROAIRE MOTORES PERKINS,250.0,MO12313,10023.63,2025-05-07 10:11:22
937,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO12113,11036.00,2025-03-28 07:39:47
1338,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11704,12236.66,2024-12-27 14:55:34
1711,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11292,10042.45,2024-09-27 15:55:31
2129,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO10871,11038.63,2024-07-11 06:44:08
2600,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO10367,11866.00,2024-04-30 15:08:05
3171,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09748,10816.48,2023-11-08 09:22:11
3616,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09266,10458.89,2023-07-26 13:46:17
3971,DA2983 FILTROAIRE MOTORES PERKINS,151.0,MO08876,11377.29,2023-05-17 16:02:46


### Corrector matches merge

### OPS DE CADA PRODUCTO

In [10]:
from fuzzywuzzy import process

# 1. Merge exacto
df_unido = productos_manu.merge(ops, on="producto", how="left", indicator=True)

# 2. Productos que no hicieron match exacto
no_match = df_unido[df_unido["_merge"] == "left_only"]["producto"].unique()

# 3. Lista de productos en OP
productos_ops = ops["producto"].dropna().unique()

# 4. Crear diccionario de equivalencias con fuzzy
mapeo = {}
for prod in no_match:
    mejor_match, score = process.extractOne(prod, productos_ops)
    if score >= 85:   # solo aceptamos matches muy altos
        mapeo[prod] = mejor_match

# 5. Reemplazar en productos_manu
productos_manu["producto_corr"] = productos_manu["producto"].replace(mapeo)

# 6. Nuevo merge usando los nombres corregidos
df_unido_corr = productos_manu.merge(ops, left_on="producto_corr", right_on="producto", how="left", indicator=True)



/home/donsson/proyectos/env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
#df_unido_corr[df_unido_corr["producto_x"].str.contains("DA2983")]

### COSTO DE CADA OP

In [12]:
from fuzzywuzzy import process

# ==============================
# 1) Calcular costo promedio real de productos_manu
# ==============================
def calcular_costo_promedio(grupo):
    if grupo["cantidad_disponible"].sum() == 0:
        return None  # evitar división por 0
    return (grupo["costo_estandar_promedio"] * grupo["cantidad_disponible"]).sum() / grupo["cantidad_disponible"].sum()

costo_promedio = (
    productos_manu.groupby("producto")
    .apply(calcular_costo_promedio)
    .reset_index(name="costo_promedio")
)

# ==============================
# 2) Merge exacto primero (ops vs productos_manu)
# ==============================
df_ops_unido = ops.merge(costo_promedio, on="producto", how="left", indicator=True)

# ==============================
# 3) Detectar OPs sin match
# ==============================
no_match_ops = df_ops_unido[df_ops_unido["_merge"] == "left_only"]["producto"].unique()

# ==============================
# 4) Lista de productos válidos (de productos_manu)
# ==============================
productos_manu_lista = costo_promedio["producto"].dropna().unique()

# ==============================
# 5) Crear diccionario de equivalencias fuzzy
# ==============================
mapeo_ops = {}
for prod in no_match_ops:
    mejor_match, score = process.extractOne(prod, productos_manu_lista)
    if score >= 85:
        mapeo_ops[prod] = mejor_match

# ==============================
# 6) Corregir los nombres en ops
# ==============================
ops["producto_corr"] = ops["producto"].replace(mapeo_ops)

# ==============================
# 7) Merge final (con fuzzy corregido)
# ==============================
ops_con_costo = ops.merge(
    costo_promedio, left_on="producto_corr", right_on="producto", how="left"
)

# ==============================
# 8) Limpiar columnas duplicadas
# ==============================

cols_a_borrar = [c for c in ["producto_x", "producto_y"] if c in ops_con_costo.columns]
ops_con_costo = ops_con_costo.drop(columns=cols_a_borrar)

# Renombrar la corregida como "producto"
if "producto_corr" in ops_con_costo.columns:
    ops_con_costo = ops_con_costo.rename(columns={"producto_corr": "producto"})


#Tarda 2 min aprox


/tmp/ipykernel_20652/787867308.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_promedio)


In [13]:
ops_con_costo[ops_con_costo["producto"].str.contains("DA2983")]

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio
127,200.0,MO13059,10804.43,2025-09-12 13:51:18,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
740,250.0,MO12313,10023.63,2025-05-07 10:11:22,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
937,150.0,MO12113,11036.00,2025-03-28 07:39:47,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
1338,150.0,MO11704,12236.66,2024-12-27 14:55:34,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
1711,150.0,MO11292,10042.45,2024-09-27 15:55:31,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
2129,150.0,MO10871,11038.63,2024-07-11 06:44:08,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
2600,200.0,MO10367,11866.00,2024-04-30 15:08:05,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
3171,300.0,MO09748,10816.48,2023-11-08 09:22:11,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
3616,300.0,MO09266,10458.89,2023-07-26 13:46:17,DA2983 FILTROAIRE MOTORES PERKINS,10686.95
3971,151.0,MO08876,11377.29,2023-05-17 16:02:46,DA2983 FILTROAIRE MOTORES PERKINS,10686.95


In [14]:
ops_con_costo['diferencia'] = ops_con_costo['costo_unitario_op'] - ops_con_costo['costo_promedio']

ops_con_costo["%diferencia"] = (ops_con_costo["diferencia"] / ops_con_costo["costo_promedio"])

ops_con_costo["%_diferencia_abs"] = (ops_con_costo["diferencia"].abs() / ops_con_costo['costo_promedio'])


In [15]:
ops_con_costo.to_excel("ops_relacionadas.xlsx")

In [16]:
ops_con_costo[ops_con_costo["costo_promedio"]==0]  # Comprobamos que no quedo ninguna OP sin el costo promedio asignado

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio,diferencia,%diferencia,%_diferencia_abs


### Df unido y corregido

In [17]:
df_unido_corr[df_unido_corr["producto_x"].str.contains("DA2983")].sample(10)

,producto_x,costo_estandar_promedio,cantidad_disponible,producto_corr,producto_y,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge
5691,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO06237,4396.14,2021-05-08 10:23:17,both
5697,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,201.0,MO04676,4173.97,2019-12-20 15:03:35,both
5690,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO06517,4485.41,2021-07-29 16:00:30,both
5687,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO08159,11963.35,2022-11-04 14:24:24,both
5692,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO05845,4239.99,2021-01-15 15:04:34,both
5701,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO03659,4879.29,2019-03-30 07:43:06,both
5702,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,201.0,MO03417,3728.81,2019-01-30 15:27:26,both
5679,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11704,12236.66,2024-12-27 14:55:34,both
5695,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,250.0,MO05075,4677.92,2020-06-11 09:54:44,both
5681,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO10871,11038.63,2024-07-11 06:44:08,both


In [18]:
# 1. Crear columna final de producto
df_unido_corr["producto"] = df_unido_corr["producto_corr"].fillna(df_unido_corr["producto_x"])

# 2. Eliminar columnas duplicadas
df_unido_corr = df_unido_corr.drop(columns=["producto_x", "producto_y", "producto_corr"], errors="ignore")

# 3. Eliminar duplicados de OP
df_unido_corr = df_unido_corr.drop_duplicates(subset=["nombre_op", "fecha_cierre_op", "producto"], keep="last")

# 4. Ordenar
df_limpio = df_unido_corr.sort_values(["producto", "fecha_cierre_op"], ascending=[True, False]).reset_index(drop=True)


In [19]:
df_limpio = df_limpio[~ (df_limpio['cantidad_disponible']==0)]  #Retirar ruido , productos sin unidades disponibles


df_limpio[df_limpio["producto"].str.contains("DA2983")]

,costo_estandar_promedio,cantidad_disponible,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge,producto
5106,10686.95,354.0,200.0,MO13059,10804.43,2025-09-12 13:51:18,both,DA2983 FILTROAIRE MOTORES PERKINS
5107,10686.95,354.0,250.0,MO12313,10023.63,2025-05-07 10:11:22,both,DA2983 FILTROAIRE MOTORES PERKINS
5108,10686.95,354.0,150.0,MO12113,11036.00,2025-03-28 07:39:47,both,DA2983 FILTROAIRE MOTORES PERKINS
5109,10686.95,354.0,150.0,MO11704,12236.66,2024-12-27 14:55:34,both,DA2983 FILTROAIRE MOTORES PERKINS
5110,10686.95,354.0,150.0,MO11292,10042.45,2024-09-27 15:55:31,both,DA2983 FILTROAIRE MOTORES PERKINS
5111,10686.95,354.0,150.0,MO10871,11038.63,2024-07-11 06:44:08,both,DA2983 FILTROAIRE MOTORES PERKINS
5112,10686.95,354.0,200.0,MO10367,11866.00,2024-04-30 15:08:05,both,DA2983 FILTROAIRE MOTORES PERKINS
5113,10686.95,354.0,300.0,MO09748,10816.48,2023-11-08 09:22:11,both,DA2983 FILTROAIRE MOTORES PERKINS
5114,10686.95,354.0,300.0,MO09266,10458.89,2023-07-26 13:46:17,both,DA2983 FILTROAIRE MOTORES PERKINS
5115,10686.95,354.0,151.0,MO08876,11377.29,2023-05-17 16:02:46,both,DA2983 FILTROAIRE MOTORES PERKINS


In [20]:
# ===========================
# 1) Función costo real
# ===========================
def calcular_costo_real(grupo):
    # ordenar las OP por fecha (descendente: más recientes primero)
    grupo = grupo.sort_values('fecha_cierre_op', ascending=False)

    cant_disp = grupo['cantidad_disponible'].iloc[0]
    total_tomado = 0
    costo_total = 0

    for _, row in grupo.iterrows():
        if total_tomado >= cant_disp:
            break

        # ignorar OP con costo nulo o cero
        if pd.isna(row['costo_unitario_op']) or row['costo_unitario_op'] == 0:
            continue

        # Tomar lo que falta o lo que tiene esta OP
        tomar = min(row['unidades_producidas'], cant_disp - total_tomado)

        costo_total += tomar * row['costo_unitario_op']
        total_tomado += tomar

    if total_tomado == 0:
        return None

    return costo_total / total_tomado


# ===========================
# 2) Costo real por producto
# ===========================
costo_real = (
    df_limpio
    .groupby('producto')
    .apply(calcular_costo_real)
    .reset_index()
)
costo_real.columns = ['producto', 'costo_real']


# ===========================
# 3) Última fecha de producción por producto
# ===========================
ultima_fecha = (
    df_limpio.groupby('producto')['fecha_cierre_op']
    .max()
    .reset_index()
)
ultima_fecha.columns = ['producto', 'ultima_fecha_produccion']


# ===========================
# 4) Construcción resumen
# ===========================
resumen = (
    df_limpio[['producto', 'costo_estandar_promedio', 'cantidad_disponible']]
    .drop_duplicates()
    .merge(costo_real, on='producto', how='left')
    .merge(ultima_fecha, on='producto', how='left')   # <-- se agrega la fecha
)

# Diferencia entre costo real y costo estándar
resumen['diferencia'] = resumen['costo_real'] - resumen['costo_estandar_promedio']

resumen["%diferencia"] = (resumen["diferencia"] / resumen["costo_estandar_promedio"])

resumen["%_diferencia_abs"] = (resumen["diferencia"].abs() / resumen["costo_estandar_promedio"])


/tmp/ipykernel_20652/3487751595.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_real)


In [21]:
resumen[resumen["costo_real"].isna()] # es un producto que tiene existencias y tiene una op pero en ella no cargaron costo unitario ejecutado fue en 2017

,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs
448,DA8021 FILTRO AIRE ASPIRADORAS INDUSTRIALES RIDGID,4856.01,7.0,NaN,2017-08-22 15:40:08,NaN,NaN,NaN


In [22]:
resumen[resumen["producto"].str.contains("DA2983", case=False, na=False)]


,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs
224,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,354.0,10464.759944,2025-09-12 13:51:18,-222.190056,-0.020791,0.020791


In [23]:
resumen.to_excel("monitoreo_costos.xlsx")

In [24]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
precio_estandar_historico = models.execute_kw(
    db, uid, password,
    'stock.change.standard.price', 'search_read',
    [[]],   # Todo
    {'fields': ['create_date', 'product_id', 'product_qty']}
)



# EQUIVALENCIAS MARCAS

In [25]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

campos = models.execute_kw(
    db, uid, password,
    'product.template', 'fields_get',
    []
)

for nombre, info in campos.items():
    if info.get('type') in ('one2many', 'many2one', 'many2many'):
        print(nombre, "->", info.get('relation'))




write_uid -> res.users
property_account_income -> account.account
website_style_ids -> product.style
product_rate -> website.product.rate
property_stock_account_output -> account.account
catalogo_valvula_alivio -> catalogo.product.valvula_alivio
company_id -> res.company
public_categ_ids -> product.public.category
pricelist_id -> product.pricelist
property_stock_account_input -> account.account
bom_ids -> mrp.bom
classprod_id -> cs.clasprod
catalogo_material_filtrante -> catalogo.product.material_filtrante
alternative_product_ids -> product.template
seller_id -> res.partner
packaging_ids -> product.packaging
pos_categ_id -> pos.category
seller_ids -> product.supplierinfo
website_message_ids -> mail.message
catalogo_marca -> catalogo.marcas
accessory_product_ids -> product.product
message_follower_ids -> res.partner
attribute_line_ids -> product.attribute.line
asset_category_id -> account.asset.category
images -> product.image
catalogo_rosca -> catalogo.product.rosca
catalogo_forma -> c

In [ ]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1) Productos básicos
productos = models.execute_kw(
    db, uid, password,
    'product.template', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'default_code']}
)
df_prod = pd.DataFrame(productos).rename(columns={
    'id': 'product_id',
    'name': 'producto_nombre',
    'default_code': 'producto_codigo'
})

# 2) Equivalencias
equivalencias = models.execute_kw(
    db, uid, password,
    'catalogo.referencia.fabricantes', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'tipo_marca', 'productos_referencia']}
)
df_eq = pd.DataFrame(equivalencias)

# 3) Expandir la lista de productos_referencia
df_eq = df_eq.explode('productos_referencia')
df_eq = df_eq.rename(columns={'name': 'equivalencia_ref'})

# 4) Limpiar tipo_marca (de lista [id, nombre] → solo nombre)
df_eq['marca'] = df_eq['tipo_marca'].apply(lambda x: x[1] if isinstance(x, list) else None)
df_eq['productos_referencia'] = df_eq['productos_referencia'].astype('Int64')

# 5) Merge con productos
df_final = df_eq.merge(df_prod, left_on='productos_referencia', right_on='product_id', how='left')
df_final = df_final.dropna()

# 6) Exportar a Excel
#df_final[['producto_codigo', 'producto_nombre', 'equivalencia_ref', 'marca']].to_excel("equivalencias.xlsx", index=False)
print("✅ Archivo 'equivalencias.xlsx' generado correctamente")


✅ Archivo 'equivalencias.xlsx' generado correctamente


In [35]:
df_final["ref_donsson"] = df_final["producto_nombre"].str.split(' ').str[0]

In [37]:
df_final = df_final[['ref_donsson', 'producto_nombre','producto_codigo', 'equivalencia_ref', 'marca']].sort_values(by=["ref_donsson"])
df_final.to_excel("equivalencias.xlsx", index=False)
df_final.head(20)

,ref_donsson,producto_nombre,producto_codigo,equivalencia_ref,marca
54908,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,AF25435,FLEETGUARD
55904,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,8082103,VOLVO
54906,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,8076195,VOLVO
54905,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,RS3740,BALDWIN
54907,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,P540388,DONALDSON
47857,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,LAF1934,LUBERFINER
47854,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,AH1197,FLEETGUARD
47855,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,CA8130,FRAM
47484,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,114880003A,FARR
47483,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,PA3493,BALDWIN
